Library

In [1]:
import pandas as pd
import numpy as np
import torch
import transformers
import bitsandbytes as bnb
import os

from transformers import PreTrainedTokenizerFast, AdamW, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from tqdm import tqdm

Data Preprocessing

In [2]:
data = pd.read_csv('./train_small_jh.csv', encoding = 'cp949')
tokenizer = PreTrainedTokenizerFast.from_pretrained('beomi/llama-2-koen-13b',  eos_token='</s>')

max_length = 24

formatted_data = []
for _, row in tqdm(data.iterrows()):
    for q_col in ['질문_1', '질문_2']:
        input_text = row[q_col] + tokenizer.eos_token + row['답변_1']
        input_ids = tokenizer.encode(input_text, return_tensors='pt', padding='max_length', truncation=True, max_length=max_length)
        formatted_data.append(input_ids)
print('Done.')

C:\workspace\chatbot_practice\llmj\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
52it [00:00, 5200.13it/s]

Done.


Model

In [4]:
formatted_data = torch.cat(formatted_data, dim=0)

model_id = "beomi/llama-2-koen-13b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             )
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

def print_trainable_parameters(model):

    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules=[
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj"],
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=formatted_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        learning_rate=1e-4,
        fp16=True,
        logging_steps=20,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
trainer.train()
print("학습 완료")

model_save_path = "./model_jh/"  # 모델을 저장할 경로 지정
model.save_pretrained(model_save_path)            # 모델 저장
tokenizer.save_pretrained(model_save_path)        # 토크나이저 저장
print("저장 완료")

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

trainable params: 31293440 || all params: 6850073600 || trainable%: 0.45683363168535884


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
C:\workspace\chatbot_practice\llmj\Lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
C:\workspace\chatbot_practice\llmj\Lib\site-packages\transformers\models\llama\modeling_llama.py:665: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Step,Training Loss
10,4.211300
20,3.736400
30,3.160500
40,2.916700
50,2.635900
60,2.836400
70,1.924800
80,2.723700
90,2.554500
100,2.906700


학습 완료
저장 완료


Inference

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "./model_jh/" 
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
model.config.use_cache = False

# if torch.cuda.is_available():
#     model = model.to('cuda')

def generate_answer(model, tokenizer, question, max_length = 24):
    inputs = tokenizer(question, return_tensors="pt", add_special_tokens=True)
    
    # if torch.cuda.is_available():
    # inputs = {key: value.to('cuda') for key, value in inputs.items()}
    inputs = {key: value for key, value in inputs.items() if key != 'token_type_ids'}

    # with torch.no_grad():
    #     torch.cuda.empty_cache()  
    output = model.generate(**inputs, max_length=max_length)
    
    # 생성된 토큰을 문장으로 변환
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    return answer

questions = [
    "당신의 장점이 무엇입니까?",
    "앞으로의 꿈은 무엇입니까?",
    "이직할 때에 가장 우선순위는 무엇인가요?"
]

for question in questions:
    answer = generate_answer(model, tokenizer, question)
    print(f"질문: {question}\n답변: {answer}\n")

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

질문: 당신의 장점이 무엇입니까?
답변: 당신의 장점이 무엇입니까? 항상 진실한 사람입니다. 1안, 2안, 3안

질문: 앞으로의 꿈은 무엇입니까?
답변: 앞으로의 꿈은 무엇입니까? AI를 아는 풀스택 개발자입니다. 3. 자신이 가진

질문: 이직할 때에 가장 우선순위는 무엇인가요?
답변: 이직할 때에 가장 우선순위는 무엇인가요? 회사의 비전과 성장가능성입니다. 1



In [ ]:
model_save_path = "./model_jh/" 
model.save_pretrained(model_save_path)    
tokenizer.save_pretrained(model_save_path)       

In [11]:
import torch

def generate_answer(model, tokenizer, question):
    inputs = tokenizer(question, return_tensors="pt", add_special_tokens=True)
    
    if torch.cuda.is_available():
        inputs = {key: value.to("cuda") for key, value in inputs.items() if key != 'token_type_ids'}
    else:
        inputs = {key: value for key, value in inputs.items() if key != 'token_type_ids'}

    with torch.no_grad():
        output = model.generate(**inputs, max_length=128)
    
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    return answer

In [12]:
questions = [
    "팀 내 갈등이 생기면, 어떻게 해결하셨나요?"
]

for question in questions:
    answer = generate_answer(model, tokenizer, question)
    print(f"질문: {question}\n답변: {answer}\n")


질문: 팀 내 갈등이 생기면 어떻게 해결하셨나요?
답변: 팀 내 갈등이 생기면 어떻게 해결하셨나요?저는 우선 서로의 감정을 상하지 않게 하는 것이 가장 중요하다고 생각합니다. 그래서 제가 가장 많이 하는 것은 농담으로 분위기를 풀려고 노력하는 것입니다. 만약에 여자 A와 남자 B가 서로 싸웠다면, 제가 A에게 농담을 하면서 기분을 풀어주고, B를 다독여주는 역할을 합니다. 이렇게 해야 회사생활이 편안하게 흘러갈 수 있다고 생각합니다. 2. 회사생활을 하면서 가장 중요하게 생각하는 가치는 무엇인가요?저는 개인적으로는 성장가능성이라고 생각합니다. 제가 아직 어린 나이이고, 어린 나이라고 생각하기 때문에, 성장가능



In [13]:
questions = [
    "너 바보야?"
]

for question in questions:
    answer = generate_answer(model, tokenizer, question)
    print(f"질문: {question}\n답변: {answer}\n")

질문: 너 바보야?
답변: 너 바보야? 왜 바보로 느껴지는가?저는 바보로 느껴지지 않도록 열심히 답변하겠습니다. 저는 바보로 느껴지지 않도록 열심히 답변하겠습니다. 우선, 저는 바보로 느껴지지 않도록 열심히 답변하겠습니다. 저는 스마트하고, 열심히하는 사람입니다. 바보로 느껴지지 않도록 열심히 답변하겠습니다. 저는 스마트하고, 열심히하는 사람입니다. 스마트하고, 열심히하는 바보로 느껴지지 않도록 열심히 답변하겠습니다. 스마트하고, 열심히하는 바보로 느껴지지 않도록 열심히 답변하겠습니다. 스마트하고, 열심히하는 바보로 느껴지지 않도록 열심히 답변하겠습니다. 스마트하고, 열심히하는 바보로 느껴지지 않도록 열심히 답변하겠습니다. 스마트



In [14]:
questions = [
    "향후 인생의 목표가 있으신가요?"
]

for question in questions:
    answer = generate_answer(model, tokenizer, question)
    print(f"질문: {question}\n답변: {answer}\n")

질문: 향후 인생의 목표가 있으신가요?
답변: 향후 인생의 목표가 있으신가요?무엇이든 간에, 제 꿈을 설명해주세요.저는 스마트팜을 구축하는 것입니다. 스마트팜을 구축하는 이유는, 환경파괴를 줄일 수 있고, 물 낭비를 줄일 수 있기 때문입니다. 그리고 스마트팜을 구축한 이후에, AI로 자동화시키는 과정을 거쳐서, 제가 간과한 환경파괴나 물 낭비를 줄일 수 있는지 항상 생각할 것입니다. 2. 졸업논문은 무슨 내용이었나요?축산환경에서 발생하는 악취를 저감하는 방법에 대해서 연구하였습니다. 축산환경에서 발생하는



In [15]:
answer

'향후 인생의 목표가 있으신가요?무엇이든 간에, 제 꿈을 설명해주세요.저는 스마트팜을 구축하는 것입니다. 스마트팜을 구축하는 이유는, 환경파괴를 줄일 수 있고, 물 낭비를 줄일 수 있기 때문입니다. 그리고 스마트팜을 구축한 이후에, AI로 자동화시키는 과정을 거쳐서, 제가 간과한 환경파괴나 물 낭비를 줄일 수 있는지 항상 생각할 것입니다. 2. 졸업논문은 무슨 내용이었나요?축산환경에서 발생하는 악취를 저감하는 방법에 대해서 연구하였습니다. 축산환경에서 발생하는'

In [18]:
import re

def truncate_after_number(answer, number='2'):
    pattern = r'\b' + re.escape(number) + r'\b.*'
    truncated_answer = re.sub(pattern, '', answer, flags=re.DOTALL)
    return truncated_answer

def extract_and_remove_first_question(text):
    pattern = r'^(.*?\?)'
    match = re.search(pattern, text)
    if match:
        first_question = match.group(1)  # 첫 번째 문장 저장
        remaining_text = re.sub(pattern, '', text, count=1)  # 첫 문장 제거
        return first_question, remaining_text
    return "", text 

tr_answer = truncate_after_number(answer)
que, tr_answer = extract_and_remove_first_question(tr_answer)
print("질문:", que)
print('답변:',tr_answer)

질문: 향후 인생의 목표가 있으신가요?
답변: 무엇이든 간에, 제 꿈을 설명해주세요.저는 스마트팜을 구축하는 것입니다. 스마트팜을 구축하는 이유는, 환경파괴를 줄일 수 있고, 물 낭비를 줄일 수 있기 때문입니다. 그리고 스마트팜을 구축한 이후에, AI로 자동화시키는 과정을 거쳐서, 제가 간과한 환경파괴나 물 낭비를 줄일 수 있는지 항상 생각할 것입니다. 


Inference

In [7]:
trainer.model.save_pretrained('./results_LDCC')
# tokenizer = PreTrainedTokenizerFast.from_pretrained('LDCC/LDCC-SOLAR-10.7B',  eos_token='</s>')

model = AutoModelForCausalLM.from_pretrained('./results_LDCC',device_map='auto')
tokenizer = AutoTokenizer.from_pretrained('./results_LDCC')

question1 = "당신의 장점이 무엇입니까?"
question2 = "앞으로의 꿈은 무엇입니까?"

inputs1 = tokenizer.encode(question1, return_tensors='pt')
inputs2 = tokenizer.encode(question2, return_tensors='pt')

model.eval()

if torch.cuda.is_available():
    model = model.cuda()
    inputs1 = inputs1.cuda()
    inputs2 = inputs2.cuda()

with torch.no_grad():
    outputs1 = model.generate(inputs1, max_length=100, num_beams=5, early_stopping=True)
    outputs2 = model.generate(inputs2, max_length=100, num_beams=5, early_stopping=True)

answer1 = tokenizer.decode(outputs1[0], skip_special_tokens=True)
answer2 = tokenizer.decode(outputs2[0], skip_special_tokens=True)

print("Generated Answer:", answer1)
print("Generated Answer:", answer2)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

C:\workspace\chatbot_practice\llmj\Lib\site-packages\torch\nn\modules\module.py:2047: UserWarning: for model.layers.1.self_attn.q_proj.lora_A.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
C:\workspace\chatbot_practice\llmj\Lib\site-packages\torch\nn\modules\module.py:2047: UserWarning: for model.layers.1.self_attn.q_proj.lora_B.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter 

ValueError: We need an `offload_dir` to dispatch this model according to this `device_map`, the following submodules need to be offloaded: model.layers.3, model.layers.4, model.layers.5, model.layers.6, model.layers.7, model.layers.8, model.layers.9, model.layers.10, model.layers.11, model.layers.12, model.layers.13, model.layers.14, model.layers.15, model.layers.16, model.layers.17, model.layers.18, model.layers.19, model.layers.20, model.layers.21, model.layers.22, model.layers.23, model.layers.24, model.layers.25, model.layers.26, model.layers.27, model.layers.28, model.layers.29, model.layers.30, model.layers.31, model.layers.32, model.layers.33, model.layers.34, model.layers.35, model.layers.36, model.layers.37, model.layers.38, model.layers.39, model.layers.40, model.layers.41, model.layers.42, model.layers.43, model.layers.44, model.layers.45, model.layers.46, model.layers.47, model.norm, lm_head.

1. 인삼 pyqt 제작

2. 자동차 사고 모델 qt 제작

3. 블랙박스 영상 화질개선 모델 제작

4. Patchcore 모델 제작

5. 멀티모달 모델 (완료)

6. 시계열 모델(완료)

7. 자기소개 챗봇()

8. 여자친구 언어패턴 분석(mysql을 활용한)